In [ ]:
import pandas as pd
import glob
import re
from zipfile import ZipFile

In [ ]:
def init_dataframe_from_zip(path, index_col):
    df_master = pd.DataFrame()
    flag = False
    files_in_folder = glob.glob(path)
    for filename in files_in_folder:    
        zip_file = ZipFile(filename)
        for text_file in zip_file.infolist():
            if text_file.filename.endswith('.csv'):
                df = pd.read_csv(zip_file.open(text_file.filename),header=0,index_col=[index_col])
            if not flag:
                df_master = df
                flag = True
            else:
                df_master = pd.concat([df_master, df])
    return df_master

csse_covid_19_daily_reports_us = init_dataframe_from_zip(r'**csv_files/csse_covid_19_daily_reports_us.zip', 'UID')
csse_covid_19_daily_reports_us.head()


In [ ]:
def init_state_social_distancing_actions(path):
    df_master = pd.DataFrame()
    flag = False
    files_in_folder = glob.glob(path)
    for filename in files_in_folder:    
        zip_file = ZipFile(filename)
        for text_file in zip_file.infolist():
            # if not text_file.filename.startswith('__MACOSX/'):
            if text_file.filename.endswith('.csv'):
                date = re.search('\d*-\d*-\d*', text_file.filename)[0]
                if date == "20201-06-01":
                    date = "2021-06-01"
                date_time_value = pd.to_datetime(date)
                df = pd.read_csv(zip_file.open(text_file.filename), sep=",", header=0)
                df["Date"] = date_time_value
                df.rename(columns = {'Unnamed: 0':'State'}, inplace = True)
                df.drop((df[df.State.isin(["United States"])].index) | (df[df.State.isnull()].index), inplace=True)
            if not flag:
                df_master = df
                flag = True
            else:
                df_master = pd.concat([df_master, df])
    df_master.set_index(["Date", "State"], inplace=True)
    df_master.sort_index(inplace=True)
    return df_master

state_social_distancing_actions = init_state_social_distancing_actions(r'**csv_files/state_social_distancing_actions.zip')

In [ ]:
def clean_state_social_distancing_actions(df):
  # TODO: Remove cells we don't want
  df = df.drop(columns=["Primary Election Postponement"])
  return df

cleaned_state_social_distancing_actions = clean_state_social_distancing_actions(state_social_distancing_actions)
cleaned_state_social_distancing_actions

In [ ]:
california_data = cleaned_state_social_distancing_actions[cleaned_state_social_distancing_actions.index.get_level_values('State').isin(['California'])]
california_data